In [1]:
!pip install torch
import torch
print(torch.__version__)

!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Change torch.__version__ of following installation command.


In [2]:
!pip install torch-geometric torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 467 kB 4.3 MB/s 
     |████████████████████████████████| 7.9 MB 4.8 MB/s 
     |████████████████████████████████| 3.5 MB 55.4 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689857 sha256=fb86b1311f40dae2d48ea99296a0d7af273526083182c3ea0928fafaa9f835a4
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
dir = "gdrive/MyDrive/USC/DS558/HW6_Neural_Symbolic/"

Preprocess data

In [7]:
import pandas as pd

# Load files. (You should change file name according to your storage path)
clicks_df = pd.read_csv(dir+'clicks.dat', header=None)
clicks_df.columns=['session_id','timestamp','item_id','category']
clicks_df.head()
 
buy_df = pd.read_csv(dir+'transaction.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']
buy_df.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,session_id,timestamp,item_id,price,quantity
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1


Sample data: Randomly sample 100K sessions (due to huge file size)

In [8]:
import numpy as np
sampled_session_id = np.random.choice(clicks_df.session_id.unique(), 100000, replace=False)
clicks_df = clicks_df.loc[clicks_df.session_id.isin(sampled_session_id)]
clicks_df.nunique()

session_id    100000
timestamp     356028
item_id        20260
category         116
dtype: int64

Encode labels with values between 0 to (num_classes - 1)

In [9]:
from sklearn.preprocessing import LabelEncoder
item_encoder = LabelEncoder()
category_encoder = LabelEncoder()
clicks_df['item_id']=item_encoder.fit_transform(clicks_df.item_id)
clicks_df['category']=category_encoder.fit_transform(clicks_df.category.apply(str))
clicks_df.head()
 
buy_df=buy_df.loc[buy_df.session_id.isin(clicks_df.session_id)]
buy_df['item_id']=item_encoder.transform(buy_df.item_id)
buy_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,session_id,timestamp,item_id,price,quantity
315,70288,2014-04-05T08:59:57.800Z,11920,15603,1
588,742,2014-04-01T18:59:24.203Z,1563,26075,1
694,210548,2014-04-04T09:07:59.826Z,14923,1256,1
695,210548,2014-04-04T09:07:59.843Z,12574,4188,1
746,351401,2014-04-04T17:45:58.229Z,14320,941,1


Create session_id - Item dictionary (Who bought which item?)

In [10]:
buy_item_dict = dict(buy_df.groupby('session_id')['item_id'].apply(list))

Define custom dataset class for train GNN
Treat each item in a session as node → All items in the same session form a graph
Items will be connected by edge.


In [11]:
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

from tqdm import tqdm
class CustomDataset(InMemoryDataset):
   def __init__(self, root, transform=None, pre_transform=None):
       super(CustomDataset, self).__init__(root, transform, pre_transform)
       self.data, self.slices = torch.load(self.processed_paths[0])
 
   @property
   def raw_file_names(self):
       return []
 
   @property
   def processed_file_names(self):
       return ['save.dataset']
 
   def download(self):
       pass
  
   def process(self):
      
       data_list = []
       # process by session_id
       grouped = clicks_df.groupby('session_id')
       for session_id, group in tqdm(grouped):
           le = LabelEncoder()
           sess_item_id = le.fit_transform(group.item_id)
           group = group.reset_index(drop=True)
           group['sess_item_id'] = sess_item_id
           node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id','category']].sort_values('sess_item_id')[['item_id','category']].drop_duplicates().values
           node_features = torch.LongTensor(node_features).unsqueeze(1)
           target_nodes = group.sess_item_id.values[1:]
           source_nodes = group.sess_item_id.values[:-1]
           edge_index = torch.tensor([source_nodes,
                                  target_nodes], dtype=torch.long)
           x = node_features
 
           if session_id in buy_item_dict:
               positive_indices = le.transform(buy_item_dict[session_id])
               label = np.zeros(len(node_features))
               label[positive_indices] = 1
           else:
               label = [0] * len(node_features)
 
           y = torch.FloatTensor(label)
           data = Data(x=x, edge_index=edge_index, y=y)
 
           data_list.append(data)
      
       data, slices = self.collate(data_list)
       torch.save((data, slices), self.processed_paths[0])
 
dataset = CustomDataset('../')


Processing...
100%|██████████| 100000/100000 [05:38<00:00, 295.83it/s]
Done!


Model

In [23]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool

num_items = clicks_df.item_id.max() +1
num_categories = clicks_df.category.max()+1
embed_dim = 128

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.item_embedding = torch.nn.Embedding(num_embeddings=num_items, embedding_dim=embed_dim)
        self.category_embedding = torch.nn.Embedding(num_embeddings=num_categories, embedding_dim=embed_dim)        
        self.conv1 = GCNConv(embed_dim * 2, embed_dim) # You can use other graph convolution layer
        self.conv2 = GCNConv(embed_dim, embed_dim)
        self.lin1 = Linear(embed_dim, 64) # You can multiple linear layers
        self.lin2 = Linear(64, 1)
        self.act1 = torch.nn.ReLU()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch  
        item_id = x[:,:,0]
        category = x[:,:,1]

        # Use item embedding & category embedding to solve the problem.
        emb_item = self.item_embedding(item_id).squeeze(1)
        emb_category = self.category_embedding(category).squeeze(1)
        x = torch.cat([emb_item, emb_category], dim=1)

        ### You can add layers or alter the model structure. See geometric documents which layer or model you can use.
        x = self.conv1(x, edge_index) # x.shape = (number of edges in the batch of graphs, representation dimension)
        x = self.act1(x)
        x = self.conv2(x, edge_index)
        x = self.act1(x)
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        ### Prediction score for each edge, range 0~1
        x = torch.sigmoid(x).squeeze(1)
        
        return x  

Dataloader

In [13]:
### add train, val, test loader.
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
### add train, val, test loader.
train_dataset, val_dataset, test_dataset = random_split(dataset,[80000,10000,10000])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Train

In [14]:
def train():
    model.train()
    criterion = torch.nn.BCELoss()
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)

        label = data.y.to(device)
        loss = criterion(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

Val

In [15]:
def val(loader):
    model.eval()
    criterion = torch.nn.BCELoss()
    loss_all = 0
    for data in loader:
        data = data.to(device)
        label = data.y.to(device)
        output = model(data)
        loss = criterion(output, label)
        loss_all += loss.item() * data.num_graphs
    return loss_all / len(train_dataset)

In [16]:
def val(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)

    return roc_auc_score(labels, predictions)

Test

In [17]:
from sklearn.metrics import roc_auc_score
def test(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)

    return roc_auc_score(labels, predictions)

In [18]:
def save_checkpoint(EPOCH, m, o, LOSS):
  torch.save({
            'epoch': EPOCH,
            'model_state_dict': m.state_dict(),
            'optimizer_state_dict': o.state_dict(),
            'loss': LOSS,
            }, dir+"model_checkpoint.md")

In [26]:
model = Net()
device = torch.device('cuda:0')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

number_of_epochs = 10 # You can change.

best_acc = 0
for epoch in range(number_of_epochs):
    train_loss = train()
    print("train_loss:",train_loss)
    val_acc = val(val_loader)
    print("val_acc:",val_acc)
    # Choose the lowest validation loss checkpoint (you can implement early stopping as well)
    if val_acc > best_acc:
      best_loss = val_acc
      save_checkpoint(epoch,model,optimizer,best_acc)
    if val_acc >= 0.67:
      break
# Load the lowest validation loss checkpoint and check the performance.


train_loss: 0.16428767057098448
val_acc: 0.6032437228815849
train_loss: 0.15302629987150432
val_acc: 0.6416688600630107
train_loss: 0.1448915508918464
val_acc: 0.6542289848494928
train_loss: 0.13900756259970368
val_acc: 0.6506411507039926
train_loss: 0.13348563874289393
val_acc: 0.6526977183691824
train_loss: 0.12972936278283595
val_acc: 0.6467826933549161
train_loss: 0.12619831148795785
val_acc: 0.6513976257850476
train_loss: 0.12226310273855924
val_acc: 0.6483218210525947
train_loss: 0.1189647931681946
val_acc: 0.6511701278832288
train_loss: 0.1156698925478384
val_acc: 0.6501206871913399


In [27]:
checkpoint = torch.load(dir+"model_checkpoint.md")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_acc = test(test_loader)

In [28]:
test_acc

0.6548630911219